In [1]:
#Import Dependencies
import os
import zipfile
import random
import shutil
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from skimage import img_as_ubyte
from skimage.util import random_noise
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

In [2]:
#Some information about the number of records :
print('Number of samples with mask in training set :',len(os.listdir('data/train/with_mask')))
print('Number of samples without mask in training set :',len(os.listdir('data/train/without_mask')))
print('Number of samples with mask in test set :',len(os.listdir('data/test/with_mask')))
print('Number of samples without mask in test set :',len(os.listdir('data/test/without_mask')))

Number of samples with mask in training set : 663
Number of samples without mask in training set : 661
Number of samples with mask in test set : 97
Number of samples without mask in test set : 97


In [3]:
#Data Agumentation :  To increase the number of images.

def h_flip(image) :
    return np.fliplr(image)

def v_flip(image) :
    return np.flipud(image)

def add_noise(image) :
    return random_noise(image)

def blur_image(image) :
    return cv2.GaussianBlur(image,(7,7),0)


transformations = {'horizontal flip' : h_flip,
                   'vertical flip' : v_flip,
                   'adding noise' : add_noise,
                   'blur image ' : blur_image}


def img_augumentation(img_path,aug_path,n_img) :
    img = []

    for im in os.listdir(img_path) :
        img.append(os.path.join(img_path,im))
         
    i = 1
    while(i <= n_img) :
        image = random.choice(img)
        original_img = mpimg.imread(image)
        transformed_img = None
        transform_key = random.choice(list(transformations)) 
        transformed_img =transformations[transform_key](original_img)

        new_img_path ="%s/aug_img_%s.jpg" %(aug_path,i)
        transformed_img = img_as_ubyte(transformed_img)
        transformed_img = cv2.cvtColor(transformed_img,cv2.COLOR_BGR2RGB)
        cv2.imwrite(new_img_path,transformed_img)
        i = i+1

In [4]:
img_augumentation('data/train/with_mask/','data/train/aug_with_mask',1000)
img_augumentation('data/train/without_mask/','data/train/aug_without_mask',1000)
img_augumentation('data/test/with_mask/','data/test/aug_with_mask',200)
img_augumentation('data/test/without_mask/','data/test/aug_without_mask',200)

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [4]:
TRAINING_DIR = "data/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))

Found 1324 images belonging to 2 classes.


In [5]:
VALIDATION_DIR = "data/val"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 142 images belonging to 2 classes.


In [6]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [7]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 133 steps, validate for 15 steps
Epoch 1/30
132/133 [============================>.] - ETA: 0s - loss: 0.6546 - acc: 0.6598WARNING:tensorflow:From C:\Users\I518759\AppData\Local\Continuum\anaconda3\envs\facemask\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model-001.model\assets
133/133 [==============================] - 110s 825ms/step - loss: 0.6524 - acc: 0.6616 - val_loss: 0.2864 - val_acc: 0.9155
Epoch 2/30
133/133 [==============================] - 100s 753ms/step - loss: 0.3635 - acc: 0.8542 - val_loss: 0.1447 - val_acc: 0.9507
Epoch 3/30
133/1

In [11]:
#! pip install pickle-mixin

  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6003 sha256=3b862e165beac1141e414def10eaa7f2536cb894230eb2b690857458d0df07a4
  Stored in directory: c:\users\i518759\appdata\local\pip\cache\wheels\82\53\b0\6f80da2d461fa5f582eb274b0158ce81d01b977cbb59a2ae6a
Successfully built pickle-mixin


In [19]:
#saving the model in pickel
import pickle

#pickle.save(model, open('facemask_model.sav', 'wb'))
model.save("face_model")

INFO:tensorflow:Assets written to: face_model\assets


In [8]:
face_clsfr=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [11]:
labels_dict={0:'with0ut_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = classifier.detectMultiScale(mini)
    
    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()
  

KeyboardInterrupt: 